In [1]:
import numpy as np
import pandas as pd
import polars as pl
import os, gc
from sklearn.preprocessing import LabelEncoder


pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [2]:
def calculate_woe_iv_categorical(feature, response):
    # Calculate the total number of events (positive responses) and non-events (negative responses)
    total_events = response.sum()
    total_non_events = response.count() - total_events
    
    # Create a new DataFrame with the feature and response values
    df = pd.DataFrame({'bin': feature.fillna('missing'), 'response': response})
    
    # Calculate the percentage of events and non-events for each bin of the feature
    bin_summary = df.groupby('bin')['response'].agg(['sum', 'count']).reset_index()
    bin_summary.columns = ['bin', 'events', 'total']
    bin_summary['non-events'] = (bin_summary['total'] - bin_summary['events']) 
    bin_summary['event_rate'] = (bin_summary['events'] / total_events)
    bin_summary['non-event_rate'] = (bin_summary['non-events'] / total_non_events) + 1e-10 # epsilon so that that the non event rate is not 0

    # Calculate the Weight of Evidence (WOE) and Information Value (IV) for each bin
    bin_summary['WOE'] = np.log1p(bin_summary['event_rate'] / bin_summary['non-event_rate'])
    bin_summary['IV'] = (bin_summary['event_rate'] - bin_summary['non-event_rate']) * bin_summary['WOE']

    # # Calculate the total Information Value (IV) for the feature
    total_IV = bin_summary['IV'].sum()
    
    return total_IV

def calculate_woe_iv_numeric(feature, response,quantiles = 25):
    # Calculate the total number of events (positive responses) and non-events (negative responses)
    total_events = response.sum()
    total_non_events = response.count() - total_events
    
    # Create a new DataFrame with the feature and response values
    df = pd.DataFrame({'feature': feature, 'response': response})
    
    # we want to support missing values
    df['bin'] = -1
    df.loc[df['feature'].notnull(),'bin'] = pd.qcut(df.loc[df['feature'].notnull(),'feature'], q=quantiles,duplicates='drop',labels=False)

    del df['feature']
    # Calculate the percentage of events and non-events for each bin of the feature
    bin_summary = df.groupby('bin')['response'].agg(['sum', 'count']).reset_index()
    bin_summary.columns = ['bin', 'events', 'total']
    bin_summary['non-events'] = (bin_summary['total'] - bin_summary['events']) 
    bin_summary['event_rate'] = (bin_summary['events'] / total_events)
    bin_summary['non-event_rate'] = (bin_summary['non-events'] / total_non_events) + 1e-10 # epsilon so that that the non event rate is not 0

    # Calculate the Weight of Evidence (WOE) and Information Value (IV) for each bin
    bin_summary['WOE'] = np.log1p(bin_summary['event_rate'] / bin_summary['non-event_rate'])
    bin_summary['IV'] = (bin_summary['event_rate'] - bin_summary['non-event_rate']) * bin_summary['WOE']

    # # Calculate the total Information Value (IV) for the feature
    total_IV = bin_summary['IV'].sum()
    
    return total_IV

def calculate_psi_categorical(old,new): 
    old = old.to_frame().fillna('missing')
    old.columns = ['bin']
    new = new.to_frame().fillna('missing')
    new.columns = ['bin']    
    
    old = old.groupby('bin').agg(count_old=('bin','count'))
    new = new.groupby('bin').agg(count_new=('bin','count'))
    
    bins = list(set(old['bin'].tolist()+new['bin'].tolist())) 
    bin_summary = pd.DataFrame(bins,columns=['bin'])
    bin_summary = bin_summary.merge(old[['bin','count_old']],on='bin',how='left')
    bin_summary = bin_summary.merge(new[['bin','count_new']],on='bin',how='left')
    bin_summary['prop_old'] = (bin_summary['count_old'].fillna(0) / len(old)) + 1e-10 # epsilon
    bin_summary['prop_new'] = (bin_summary['count_new'].fillna(0) / len(new)) + 1e-10 # epsilon

    return np.sum((bin_summary['prop_old'] - bin_summary['prop_new']) * np.log(bin_summary['prop_old']/bin_summary['prop_new']))

def calculate_psi_numeric(old,new,q=10): 

    old = old.to_frame()
    old.columns = ['val']
    new = new.to_frame()
    new.columns = ['val']
    
    # set up initial bins for missing values
    old['bin'] = -1
    new['bin'] = -1
    
    
    # return 0 in the event that theres less than 3 unique bin across both
    if (old['val'].fillna(-9999).nunique() + new['val'].fillna(-9999).nunique()) <= 2:
        return np.nan
    else: 
        # assign each value to a quantile 
        old.loc[old['val'].notnull(),'bin'] = pd.qcut(old.loc[old['val'].notnull(),'val'], q=q,duplicates='drop',labels=False)
        new.loc[new['val'].notnull(),'bin'] = pd.qcut(new.loc[new['val'].notnull(),'val'], q=q,duplicates='drop',labels=False)
        
        old = old.groupby('bin').agg(count_old=('bin','count'))
        new = new.groupby('bin').agg(count_new=('bin','count'))
        
        
        bins = list(set(old['bin'].tolist()+new['bin'].tolist())) 
        bin_summary = pd.DataFrame(bins,columns=['bin'])
        bin_summary = bin_summary.merge(old[['bin','count_old']],on='bin',how='left')
        bin_summary = bin_summary.merge(new[['bin','count_new']],on='bin',how='left')
        bin_summary['prop_old'] = (bin_summary['count_old'].fillna(0) / len(old)) + 1e-10 # epsilon
        bin_summary['prop_new'] = (bin_summary['count_new'].fillna(0) / len(new)) + 1e-10 # epsilon
    
        return np.sum((bin_summary['prop_old'] - bin_summary['prop_new']) * np.log(bin_summary['prop_old']/bin_summary['prop_new']))

# Preprocessing

[Data Info](https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability/data) <br>
[Discussion on how the data is setup](https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability/discussion/473950) <br>
[Starter Notebook](https://www.kaggle.com/code/jetakow/home-credit-2024-starter-notebook)
* depth=0 - These are static features directly tied to a specific case_id.
* depth=1 - Each case_id has an associated historical record, indexed by num_group1.
* depth=2 - Each case_id has an associated historical record, indexed by both num_group1 and num_group2.

In [3]:
class Aggregator:
    def __init__(self,numeric_cols,string_cols,date_cols,criteria):
        self.numeric_cols = numeric_cols
        self.string_cols  = string_cols
        self.date_cols    = date_cols
        self.criteria = criteria
        
    def num_expr(self,col):
        
        expr_max    = [pl.max(col).alias(f"{col}_MAX_{self.criteria}")]
        expr_min    = [pl.min(col).alias(f"{col}_MIN_{self.criteria}")]
        expr_last   = [pl.last(col).alias(f"{col}_LAST_{self.criteria}")]
        expr_mean   = [pl.mean(col).alias(f"{col}_MEAN_{self.criteria}")]
        expr_median = [pl.median(col).alias(f"{col}_MEDIAN_{self.criteria}")]
        expr_var    = [pl.var(col).alias(f"{col}_VAR_{self.criteria}")]

        return expr_max + expr_last + expr_mean + expr_median + expr_var + expr_min

    def date_expr(self,col):
        expr_max    = [pl.max(col).alias(f"{col}_MAX_{self.criteria}")]
        expr_last   = [pl.last(col).alias(f"{col}_LAST_{self.criteria}")]
        expr_mean   = [pl.mean(col).alias(f"{col}_MEAN_{self.criteria}")]

        return expr_max + expr_last + expr_mean 

    def str_expr(self,col):
        expr_max    = [pl.max(col).alias(f"{col}_MAX_{self.criteria}")]
        expr_last   = [pl.last(col).alias(f"{col}_LAST_{self.criteria}")]
        return expr_max + expr_last 

    def count_expr(self,col):
        expr_max    = [pl.max(col).alias(f"{col}_MAX_{self.criteria}")]

        return expr_max

    def get_exprs(self,df):
        expr = []
        new_date_cols = []
        new_str_cols = []
        for col in df.columns:
            if 'num_group' in col:
                expr.extend(self.count_expr(col))
            elif col in self.numeric_cols:
                expr.extend(self.num_expr(col))
            elif col in self.string_cols:
                new_str_cols.extend([f"{col}_MAX_{self.criteria}",f"{col}_LAST_{self.criteria}"])
                expr.extend(self.str_expr(col))
            elif col in self.date_cols:
                new_date_cols.extend([f"{col}_MAX_{self.criteria}",f"{col}_LAST_{self.criteria}",f"{col}_MEAN_{self.criteria}"])
                expr.extend(self.date_expr(col))
        
        return expr, new_date_cols, new_str_cols

In [4]:
def filter_cols(df: pl.DataFrame) -> pl.DataFrame:
    """
    Filters columns in the DataFrame based on null percentage and unique values for string columns.

    Args:
    - df (pl.DataFrame): Input DataFrame.

    Returns:
    - pl.DataFrame: DataFrame with filtered columns.
    """
    for col in df.columns:
        if col not in ["case_id", "year", "month", "week_num", "target"]:
            null_pct = df[col].is_null().mean()

            if null_pct > 0.97:
                df = df.drop(col)
                print(f"dropped column {col} because too many nulls")
#     for col in df.columns:
#         if (col not in ["case_id", "year", "month", "week_num", "target"]) & (
#             df[col].dtype == pl.String
#         ):
#             freq = df[col].n_unique()

#             if (freq > 200) | (freq == 1):
#                 df = df.drop(col)
#                 print(f"dropped column {col} because of category size")

    return df

In [5]:
def reduce_polars_memory_usage(df: pl.DataFrame) -> pl.DataFrame:
    """
    Reduces memory usage of a DataFrame by converting column types.

    Args:
    - df (pl.DataFrame): DataFrame to optimize.
    - name (str): Name of the DataFrame.

    Returns:
    - pl.DataFrame: Optimized DataFrame.
    """
    og_mem = round(df.estimated_size('mb'), 4)

    int_types = [
        pl.Int8,
        pl.Int16,
        pl.Int32,
        pl.Int64,
        pl.UInt8,
        pl.UInt16,
        pl.UInt32,
        pl.UInt64,
    ]
    float_types = [pl.Float32, pl.Float64]

    for col in df.columns:
        if col == 'case_id':
            continue
        col_type = df[col].dtype
        if col_type in int_types + float_types:
            c_min = df[col].min()
            c_max = df[col].max()

            if c_min is not None and c_max is not None:
                if col_type in int_types:
                    if c_min >= 0:
                        if (
                            c_min >= np.iinfo(np.uint8).min
                            and c_max <= np.iinfo(np.uint8).max
                        ):
                            df = df.with_columns(df[col].cast(pl.UInt8))
                        elif (
                            c_min >= np.iinfo(np.uint16).min
                            and c_max <= np.iinfo(np.uint16).max
                        ):
                            df = df.with_columns(df[col].cast(pl.UInt16))
                        elif (
                            c_min >= np.iinfo(np.uint32).min
                            and c_max <= np.iinfo(np.uint32).max
                        ):
                            df = df.with_columns(df[col].cast(pl.UInt32))
                        elif (
                            c_min >= np.iinfo(np.uint64).min
                            and c_max <= np.iinfo(np.uint64).max
                        ):
                            df = df.with_columns(df[col].cast(pl.UInt64))
                    else:
                        if (
                            c_min >= np.iinfo(np.int8).min
                            and c_max <= np.iinfo(np.int8).max
                        ):
                            df = df.with_columns(df[col].cast(pl.Int8))
                        elif (
                            c_min >= np.iinfo(np.int16).min
                            and c_max <= np.iinfo(np.int16).max
                        ):
                            df = df.with_columns(df[col].cast(pl.Int16))
                        elif (
                            c_min >= np.iinfo(np.int32).min
                            and c_max <= np.iinfo(np.int32).max
                        ):
                            df = df.with_columns(df[col].cast(pl.Int32))
                        elif (
                            c_min >= np.iinfo(np.int64).min
                            and c_max <= np.iinfo(np.int64).max
                        ):
                            df = df.with_columns(df[col].cast(pl.Int64))
                elif col_type in float_types:
                    if (
                        c_min > np.finfo(np.float32).min
                        and c_max < np.finfo(np.float32).max
                    ):
                        df = df.with_columns(df[col].cast(pl.Float32))

    print(
        f"Memory of polars dataframe went from {og_mem}MB to {round(df.estimated_size('mb'), 4)}MB."
    )

    return df

In [6]:
def extract_lowercase(s):
    # Initialize an empty result string
    result = ""

    # Loop through each character in the string
    for char in s:
        # Check if the character is lowercase
        if char.islower() or char == '_' or char.isnumeric():
            result += char
        # Break the loop if a non-lowercase character is encountered (if desired)
        elif result:
            break

    return result

In [7]:
class DatasetBuilder:
    """ This class is used to create the dataset """
    def __init__(self, 
                 n_samples   = None, 
                 parent_path = "/kaggle/input/home-credit-credit-risk-model-stability",
                ):
        


        self.parent_path = parent_path
        self.n_samples = n_samples

        self.feat_info = pd.read_csv(f"{parent_path}/feature_definitions.csv")
        self.date_cols = []
        self.string_cols = []
        
        self.run()

    def explain_feat(self,feat_name:str):
        assert feat_name in self.feat_info['Variable'].unique(), "feature not found in feature info dataframe"
        return self.feat_info[self.feat_info['Variable']==feat_name]['Description'].values[0]

    def set_table_dtypes(self,df):
        for col in df.columns:
                    
            if col in ["case_id"]:
                df = df.with_columns(pl.col(col).cast(pl.Int32))
            elif col in  ["WEEK_NUM", "num_group1", "num_group2", "target"]:
                df = df.with_columns(pl.col(col).cast(pl.UInt16))            
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))                
            elif (col[-1] in ("M",)) or (col in self.string_cols):
                df = df.with_columns(pl.col(col).cast(pl.String))
                if col not in self.string_cols:
                    self.string_cols.append(col)
            elif col[-1] in ("L","T"): # we dont know the transform needed, just going to assume its either float and if not, then string
                try:
                    df = df.with_columns(pl.col(col).cast(pl.Float64))
                except:
                    df = df.with_columns(pl.col(col).cast(pl.String))
                    if col not in self.string_cols:
                        self.string_cols.append(col) 
                    continue
                
            elif col[-1] in ("D",) or (col in self.date_cols):
                df = df.with_columns(pl.col(col).cast(pl.Date))
                if col not in self.date_cols:
                    self.date_cols.append(col)
        return df

    def feature_engineer_dates(self,df,date_cols=None):
        if date_cols is None:
            date_cols = self.date_cols
        if 'date_decision' not in df.columns:
            df = df.join(self.df[['case_id','date_decision']],on='case_id')
        for col in date_cols:
            if col in df.columns:
                df = df.with_columns((pl.col("date_decision") - pl.col(col)).dt.total_days().alias(f'{col}_DAYS_SINCE'))
                df = df.drop(col)

        if 'date_decision' in df.columns:
            df = df.drop('date_decision')
            
        return df
  
    
    def create_base_dataset(self):
        
        # load in the training dataset 
        if self.n_samples is not None:
            train = pl.read_parquet(f"{self.parent_path}/parquet_files/train/train_base.parquet") \
            .sample(n=self.n_samples).with_columns(pl.lit('train').alias('partition'))
        else:
            train = pl.read_parquet(f"{self.parent_path}/parquet_files/train/train_base.parquet") \
            .with_columns(pl.lit('train').alias('partition'))
        
        # load in the test dataset
        test =  pl.read_parquet(f"{self.parent_path}/parquet_files/test/test_base.parquet")\
                .with_columns(pl.lit(0).alias('target'))\
                .with_columns(pl.lit('test').alias('partition'))        
        
        
        
        
        
        # concat train and test
        self.df = reduce_polars_memory_usage(pl.concat([train,test],how='vertical_relaxed').pipe(self.set_table_dtypes))
        
        # get all case_ids
        self.train_case_ids = train.get_column('case_id').to_list()
        self.test_case_ids  = test.get_column('case_id').to_list()
        
        # store base cols
        self.base_df_cols = self.df.columns
        
        del train
        del test
        gc.collect()

    def read_in_files_with_criteria(self, criteria:str):
        print(f"processing criteria {criteria}...")
        train_df  = pl.concat([pl.scan_parquet(f"{self.parent_path}/parquet_files/train/{x}", low_memory=True, rechunk=True)
                       for x in os.listdir(f"{self.parent_path}/parquet_files/train") if (criteria in x)],how='vertical_relaxed')
        test_df  =  pl.concat([pl.scan_parquet(f"{self.parent_path}/parquet_files/test/{x}", low_memory=True, rechunk=True)
                       for x in os.listdir(f"{self.parent_path}/parquet_files/test") if (criteria in x)],how='vertical_relaxed')
        

        # being in train partition doesnt gaurentee it is in the test partition, so we have to ensure it 
        columns_in_common = list(set(train_df.columns).intersection(set(test_df.columns)))

        df = pl.concat([train_df.select(columns_in_common),
                         test_df.select(columns_in_common)],how='vertical_relaxed') 
            
        del train_df
        del test_df 
        gc.collect()
        
        df = df.collect().pipe(self.set_table_dtypes).filter(pl.col('case_id').is_in(self.train_case_ids+self.test_case_ids))

        return df
        
    def optimize_polars_df(self,df):
        return reduce_polars_memory_usage(filter_cols(df))
       
        

    def evaluate_features(self,df:pl.DataFrame,
                          stability_scoring=False):
        """
        1) calculates weight of evidence * information value for measuring predictive power
        
        """
        feats = [x for x in df.columns if x not in self.base_df_cols]

        df = df.filter(pl.col("case_id").is_in(self.train_case_ids))
        n_row = len(self.df)
        if 'target' not in df.columns:
            df = df.join(self.df[['case_id','target']],on='case_id')
        
        # predictive power - woe*iv
        woeivs  = []
        for col in feats:
            if df[col].dtype == pl.String:
                woeiv = calculate_woe_iv_categorical(df[col].to_pandas(), df['target'].to_pandas())
                woeivs.append(woeiv)
            else:
                woeiv = calculate_woe_iv_numeric(df[col].to_pandas(), df['target'].to_pandas())
                woeivs.append(woeiv)
        

        feature_scores = pd.DataFrame(feats,columns=['feature'])
        feature_scores['categorical'] = feature_scores['feature'].isin(self.string_cols)
        feature_scores['prop_null'] = feature_scores['feature'].apply(lambda feat: df[feat].to_pandas().isna().sum()) / n_row
        feature_scores['woe_iv'] = woeivs       
    
        return feature_scores
        
    
    def select_features(self,df,score="woe_iv",threshold=0.05,dedup_agg=False):
        feature_scores = self.evaluate_features(df)
        start_n = len(feature_scores)
        
        if dedup_agg:
            feature_scores['feature_base_col'] = feature_scores['feature'].apply(extract_lowercase)
            feature_scores = feature_scores.sort_values(['feature_base_col',score],ascending=[True,False]).drop_duplicates(subset=['feature_base_col'])
        
        chosen_features = feature_scores[feature_scores[score]>=threshold]['feature'].unique().tolist()
        print(f"selected {len(chosen_features)}/{start_n} features for the model dataset")
        del feature_scores
        return chosen_features

    
    def to_pandas(self,df_data):
        df_data = df_data.to_pandas()
        cat_cols = [x for x in df_data.columns if (x in self.string_cols) and (x not in self.base_df_cols)]
        df_data[cat_cols] = df_data[cat_cols].astype("category")
        df_data = self.reduce_pandas_mem_usage(df_data)
        return df_data, cat_cols

    def reduce_pandas_mem_usage(self,df):
        """ iterate through all the columns of a dataframe and modify the data type
            to reduce memory usage.        
        """
        start_mem = df.memory_usage().sum() / 1024**2
        print('Memory usage of pandas dataframe is {:.2f} MB'.format(start_mem))

        for col in [x for x in df.columns if x not in self.base_df_cols]:
            col_type = df[col].dtype
            if str(col_type)=="category":
                continue
                
            else:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)  
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)


        end_mem = df.memory_usage().sum() / 1024**2
        print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

        return df
     
    def process_depth0(self):
        """
        These files can be used as is except for the dates, so just collect them, do feature engineering on the dates, then 
        throw out the date columns, grab top k features, join back to base
        """
        depth0_criterias = ["static_0","static_cb_0"]

        for criteria in depth0_criterias:
            df = self.read_in_files_with_criteria(criteria)
            df = self.optimize_polars_df(df)
            df = self.feature_engineer_dates(df)
            depth0_feats = self.select_features(df,score="woe_iv")
            self.df = self.df.join(df[['case_id']+depth0_feats], on='case_id', how='left')   
        
        del df
        gc.collect()

    def process_depth1(self):
        """
        These files have one group; collect them, auto aggregate, do feature engineering on the dates,
        throw out the date columns, grab top k features, join back to base
        """
        depth1_criterias = ["applprev_1","other_1",
                            "tax_registry_a_1","tax_registry_b_1","tax_registry_c_1",
                            "credit_bureau_a_1","credit_bureau_b_1",
                            "deposit_1","person_1","debitcard_1"]
        
        # all groups
        for criteria in depth1_criterias:
            df = self.df[['case_id','target','date_decision']]
            
            criteria_df = self.read_in_files_with_criteria(criteria)
            criteria_df = self.optimize_polars_df(criteria_df)
            aggr = Aggregator([x for x in criteria_df.columns if x not in self.string_cols+self.date_cols+self.base_df_cols],
                              self.string_cols,self.date_cols,
                              f"{criteria.upper()}_DEPTH1_ALL")
            agg_expr, agg_dt_cols, agg_str_cols = aggr.get_exprs(criteria_df)
 
            criteria_df = criteria_df.group_by("case_id").agg(agg_expr)
            df = df.join(criteria_df, on=['case_id'], how='left')
            df = self.feature_engineer_dates(df,date_cols=agg_dt_cols)    

            feats = self.select_features(df,score="woe_iv",dedup_agg=True)
            
            if len(feats)>0:
                self.string_cols.extend([x for x in agg_str_cols if x in feats])
                self.df = self.df.join(df[['case_id']+feats], on='case_id', how='left') 
 
            del criteria_df
            del df
            gc.collect()

    def process_depth2(self):
        """
        For now, just approach it like depth 2
        """
        depth2_criterias = ["applprev_2","person_2","credit_bureau_b_2"] # "credit_bureau_a_2",
        

        for criteria in depth2_criterias:
            df = self.df[['case_id','target','date_decision']]
            # all groups
            criteria_df = self.read_in_files_with_criteria(criteria)
            criteria_df = self.optimize_polars_df(criteria_df)
            aggr = Aggregator([x for x in criteria_df.columns if x not in self.string_cols+self.date_cols+self.base_df_cols],
                              self.string_cols,self.date_cols,
                              f"{criteria.upper()}_DEPTH2_ALL")
            agg_expr, agg_dt_cols, agg_str_cols = aggr.get_exprs(criteria_df)
 
            criteria_df = criteria_df.group_by("case_id").agg(agg_expr)
            df = df.join(criteria_df, on=['case_id'], how='left')
            df = self.feature_engineer_dates(df,date_cols=agg_dt_cols)    
            feats = self.select_features(df,score="woe_iv",dedup_agg=True)
            if len(feats)>0:
                self.string_cols.extend([x for x in agg_str_cols if x in feats])
                self.df = self.df.join(df[['case_id']+feats], on='case_id', how='left') 
 
            del criteria_df
            del df
            gc.collect()            
            

    def run(self):
        self.create_base_dataset()
        self.process_depth0()
        self.process_depth1()
        self.process_depth2()        
        
    def get_datasets(self):
        df,cat_cols = self.to_pandas(self.df)

        del self.df
        gc.collect()
        
        return {"train":df[df['partition']=='train'].reset_index(drop=True), 
                "test": df[df['partition']=='test'].reset_index(drop=True), 
                "features": [x for x in df.columns if x not in self.base_df_cols],
                "cat_features": cat_cols}
    


In [ ]:
ds = DatasetBuilder().get_datasets()

Memory of polars dataframe went from 36.3986MB to 27.6629MB.
processing criteria static_0...
dropped column clientscnt_136L because too many nulls
dropped column lastotherlnsexpense_631A because too many nulls
dropped column lastdependentsnum_448L because too many nulls
dropped column lastrepayingdate_696D because too many nulls
dropped column payvacationpostpone_4187118D because too many nulls
dropped column lastotherinc_902A because too many nulls
dropped column equalityempfrom_62L because too many nulls
dropped column isbidproductrequest_292L because too many nulls
dropped column interestrategrace_34L because too many nulls
Memory of polars dataframe went from 1715.8248MB to 976.2047MB.


In [ ]:
print(ds['train'].shape)
ds['train']['target'].value_counts(normalize=True)

In [ ]:
# del DSBuilder
# gc.collect()

In [ ]:
ds['train']

# Training LGBM

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split,StratifiedGroupKFold
import lightgbm as lgb 
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK
from hyperopt.pyll import scope
from functools import partial

In [ ]:
def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
    gini_in_time = base.loc[:, ["WEEK_NUM", "target", "score"]]\
        .sort_values("WEEK_NUM")\
        .groupby("WEEK_NUM")[["target", "score"]]\
        .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1).tolist()
    
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

In [ ]:
def get_base_params():
    base_params = {
        'boosting_type':'gbdt',
        'random_state': 117,
        'objective': 'binary',
        'metric': 'auc',
        'extra_trees':True,
        'verbose': -1,
        'max_bin': 64,
        'device_type': 'gpu'
        
    }
    return base_params

In [ ]:
search_space_setup = {
    'feature_fraction': hp.uniform('colsample_bynode', 0.3, 0.8),
    'max_depth': scope.int(hp.uniform('max_depth', 5, 20)),
    'l1_regularization': hp.loguniform('l1_regularization', np.log(.001), np.log(100)),
    'l2_regularization':hp.loguniform('l2_regularization',np.log(.001), np.log(100)),
    'cat_l2': hp.loguniform('cat_l2', np.log(.001), np.log(100)),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.3, 0.8),
    'bagging_freq': scope.int(hp.uniform('bagging_freq', 0, 5)),
    'learning_rate' : hp.loguniform('learning_rate', np.log(0.001), np.log(.5)),
    'n_estimators':scope.int(hp.uniform('n_estimators', 500, 1500)),
    

}
search_space = get_base_params()
for k,v in search_space_setup.items():
    search_space[k] = v

In [ ]:
# do splits ahead of time to improve trial speed
k = 5

lgbtrain = lgb.Dataset(ds['train'].loc[:,ds['features']], label=ds['train'].loc[:,'target'])
test_X = ds['test'][ds['features']]
skf = StratifiedGroupKFold(n_splits=k)
splits = skf.split(ds['train'].loc[:,ds['features']],ds['train']['target'],groups = ds['train']['WEEK_NUM'])

submission = ds['test'][['case_id']]

In [ ]:
del ds
gc.collect()

In [ ]:
def trial_fn(params,
             splits = None,
             dataset = None):
    
    num_boost_round = params.pop('n_estimators')

    cv_results = lgb.cv(
        params,
        dataset,
        num_boost_round=num_boost_round,
        folds=splits,
        seed = 117,
        callbacks = [lgb.early_stopping(20)]
    ) 
    
    print(cv_results)
    score = np.mean(cv_results['auc-mean'])
    return {"status": STATUS_OK, "loss": -score} # always minimizes

In [ ]:
best_params = fmin(fn=partial(trial_fn, splits = splits, dataset = lgbtrain),
                    space=search_space,
                    algo=tpe.suggest,
                    max_evals=100,
                    timeout=60*60*2 # seconds
                  )
int_params = ['max_depth','n_estimators','bagging_freq']
bestp = get_base_params()
for k,v in best_params.items():
    if k in int_params:
        bestp[k] = int(v)
    else:
        bestp[k] = v
bestp

In [ ]:
gbm = lgb.train(
    n_estimators,
    lgbtrain 
)

# Submission


In [ ]:
submission['score'] = mod.predict(test_X)

In [ ]:
submission.to_csv('submission.csv', index=False)
submission.head()